## Fully automated Python workflow for San Francisco. 

It will:

Download the .osm.pbf data.

Load it with pyrosm or quackosm.

Extract parks, streets, and buildings as separate GeoDataFrames.

Save them as GeoPackage files, ready to load into QGIS.

In [2]:
# Install necessary packages if not already installed
# %pip install pyrosm quackosm geopandas

In [4]:
import os
import quackosm as qosm
from pyrosm import OSM, get_data
import geopandas as gpd

# -------------------------------
# 1️⃣ Download San Francisco OSM data (Geofabrik)
# -------------------------------
# You can replace this with a local path if you already have the .osm.pbf
import requests

sf_pbf_url = "https://download.geofabrik.de/north-america/us/california/norcal-latest.osm.pbf"
sf_pbf_file = "san-francisco.osm.pbf"

with requests.get(sf_pbf_url, stream=True) as r:
    r.raise_for_status()  # stop if download fails
    with open(sf_pbf_file, "wb") as f:
        for chunk in r.iter_content(chunk_size=1024*1024):
            if chunk:
                f.write(chunk)

    print("Download complete!")

# -------------------------------
# 2️⃣ Load OSM with quackosm
# -------------------------------
print("Loading OSM data with quackosm...")
gdf = qosm.convert_pbf_to_geodataframe(sf_pbf_file)
print("OSM loaded:", len(gdf), "features")

# -------------------------------
# 3️⃣ Extract features
# -------------------------------
# Parks
# Ensure the keys exist
parks = gdf[
    gdf.get("leisure", "").fillna("").isin(["park"]) |
    gdf.get("landuse", "").fillna("").isin(["recreation_ground"])
]

print("Parks:", len(parks))

# Buildings
buildings = gdf[gdf["building"].notna()]
print("Buildings:", len(buildings))

# Streets (highways)
streets = gdf[gdf["highway"].notna()]
print("Streets:", len(streets))

# -------------------------------
# 4️⃣ Save to GeoPackage for QGIS
# -------------------------------
output_folder = "san_francisco_layers"
os.makedirs(output_folder, exist_ok=True)

parks.to_file(os.path.join(output_folder, "parks.gpkg"), layer="parks", driver="GPKG")
buildings.to_file(os.path.join(output_folder, "buildings.gpkg"), layer="buildings", driver="GPKG")
streets.to_file(os.path.join(output_folder, "streets.gpkg"), layer="streets", driver="GPKG")

print(f"Saved layers to folder: {output_folder}")


Output()

Download complete!
Loading OSM data with quackosm...


Encountered MemoryError during operation. Retrying with lower number of rows per group (4000000).

libc++abi: terminating due to uncaught exception of type duckdb::OutOfMemoryException: {"exception_type":"Out of Memory","exception_message":"failed to pin block of size 256.0 KiB (12.7 GiB/12.7 GiB used)\n\nPossible solutions:\n* Reducing the number of threads (SET threads=X)\n* Disabling insertion-order preservation (SET preserve_insertion_order=false)\n* Increasing the memory limit (SET memory_limit='...GB')\n\nSee also https://duckdb.org/docs/stable/guides/performance/how_to_tune_workloads"}


Encountered MemoryError during operation. Retrying with lower number of rows per group (1000000).

Finished operation in 0:06:24

OSM loaded: 9060584 features


AttributeError: 'str' object has no attribute 'fillna'

In [ ]:
import quackosm as qosm
gdf = qosm.convert_pbf_to_geodataframe("san-francisco.osm.pbf")


Output()

IOException: IO Error: Cannot open file "san-francisco_quackosm.osm.pbf": No such file or directory

✅ How to use in QGIS

Open QGIS.

Layer → Add Layer → Add Vector Layer → GeoPackage.

Select parks.gpkg, buildings.gpkg, and streets.gpkg.

The layers are ready to style:

Parks → green fill

Buildings → gray fill

Streets → black or colored lines

🔹 Notes

quackosm.convert_pbf_to_geodataframe() handles all OSM tags.

You can filter by other tags (waterways, railways, landuse, etc.) the same way.

### Let’s make an enhanced Python workflow for Paris OSM data that:

Downloads the PBF (with automatic size check).

Loads it using quackosm.

Extracts parks, buildings, and streets.

Saves them as GeoPackages.

Applies nice default styles (colors, line widths, transparency) in QGIS-ready form.

In [ ]:
# Enhanced Paris OSM workflow
# --------------------------------------------------
# Install dependencies if needed:
# !pip install quackosm geopandas

import os
import requests
import quackosm as qosm
import geopandas as gpd

# -------------------------------
# 1️⃣ Download Paris OSM data (with size check)
# -------------------------------
sf_pbf_url = "https://download.geofabrik.de/europe/france/ile-de-france.osm.pbf"
sf_pbf_file = "ile-de-france.osm.pbf"
expected_size_bytes = 96810294  # ~96 MB

download_needed = True
if os.path.exists(sf_pbf_file):
    actual_size = os.path.getsize(sf_pbf_file)
    if actual_size >= expected_size_bytes:
        download_needed = False
    else:
        print("Existing file incomplete. Redownloading...")
        os.remove(sf_pbf_file)

if download_needed:
    print("Downloading Paris OSM data (~96MB)...")
    with requests.get(sf_pbf_url, stream=True) as r:
        r.raise_for_status()
        with open(sf_pbf_file, "wb") as f:
            for chunk in r.iter_content(chunk_size=1024*1024):
                if chunk:
                    f.write(chunk)
    print("Download complete!")

# -------------------------------
# 2️⃣ Load OSM with quackosm
# -------------------------------
print("Loading OSM data with quackosm...")
gdf = qosm.convert_pbf_to_geodataframe(sf_pbf_file)
print("OSM loaded:", len(gdf), "features")

# -------------------------------
# 3️⃣ Extract feature layers
# -------------------------------
# Ensure the keys exist
parks = gdf[
    gdf.get("leisure", "").fillna("").isin(["park"]) |
    gdf.get("landuse", "").fillna("").isin(["recreation_ground"])
]

buildings = gdf[gdf["building"].notna()]
streets = gdf[gdf["highway"].notna()]

print("Features extracted:")
print("Parks:", len(parks))
print("Buildings:", len(buildings))
print("Streets:", len(streets))

# -------------------------------
# 4️⃣ Save to GeoPackage with styling hints
# -------------------------------
output_folder = "paris_layers"
os.makedirs(output_folder, exist_ok=True)

# Function to save with styling metadata
def save_gpkg_with_style(gdf, filename, layer_name, style):
    path = os.path.join(output_folder, filename)
    gdf.to_file(path, layer=layer_name, driver="GPKG")
    # Save simple style metadata in a companion JSON (optional)
    import json
    style_file = path.replace(".gpkg", "_style.json")
    with open(style_file, "w") as f:
        json.dump(style, f, indent=2)

# Define simple style dictionaries
park_style = {"fill_color": "#78C679", "outline_color": "#4FCE30", "fill_opacity": 0.5}
building_style = {"fill_color": "#987E7E", "outline_color": "#666666", "fill_opacity": 1.0}
street_style = {"line_color": "#271717", "line_width": 0.8}

save_gpkg_with_style(parks, "parks.gpkg", "parks", park_style)
save_gpkg_with_style(buildings, "buildings.gpkg", "buildings", building_style)
save_gpkg_with_style(streets, "streets.gpkg", "streets", street_style)

print(f"Layers saved to '{output_folder}' with styling hints. Ready for QGIS!")


✅ How it works

Automatic download check: ensures the .osm.pbf file is complete before loading.

Feature extraction: separates parks, buildings, streets into GeoDataFrames.

GeoPackage export: saves each layer in a single .gpkg file.

Styling hints: writes a companion JSON with colors and opacities. QGIS doesn’t read this automatically, but you can use it as reference for styling layers quickly.

💡 Styling in QGIS

Load each .gpkg layer.

Open Layer → Properties → Symbology.

Apply the colors from the JSON:

Parks → green fill with transparency

Buildings → gray fill

Streets → thin dark lines

### fancier, fully automated workflow for QGIS using PyQGIS. This version:

Downloads San Francisco OSM PBF (with file size check).

Loads it using quackosm into GeoDataFrames.

Extracts parks, buildings, streets safely (avoiding KeyErrors).

Saves them as GeoPackages.

Automatically loads them into QGIS and applies nice styles with PyQGIS.

⚠️ This needs to be run inside the QGIS Python console or as a QGIS standalone script.

In [ ]:
# ------------------------------------------------------
# Fully Automated San Francisco OSM Workflow with PyQGIS
# ------------------------------------------------------

import os
import requests
import quackosm as qosm
import geopandas as gpd

# -------------------------------
# 1️⃣ Download Ibizs OSM data
# -------------------------------
sf_pbf_url = "https://download.geofabrik.de/europe/spain/islas-baleares.osm.pbf"
sf_pbf_file = "islas-baleares.osm.pbf"
expected_size_bytes = 96810294  # ~96 MB

if not os.path.exists(sf_pbf_file) or os.path.getsize(sf_pbf_file) < expected_size_bytes:
    print("Downloading Ibiza OSM data (~96MB)...")
    with requests.get(sf_pbf_url, stream=True) as r:
        r.raise_for_status()
        with open(sf_pbf_file, "wb") as f:
            for chunk in r.iter_content(chunk_size=1024*1024):
                if chunk:
                    f.write(chunk)
    print("Download complete!")

# -------------------------------
# 2️⃣ Load OSM with quackosm
# -------------------------------
print("Loading OSM data with quackosm...")
gdf = qosm.convert_pbf_to_geodataframe(sf_pbf_file)
print("OSM loaded:", len(gdf), "features")

# -------------------------------
# 3️⃣ Extract features safely
# -------------------------------
def get_column_safe(gdf, col):
    """Return column if exists, otherwise empty string"""
    if col in gdf.columns:
        return gdf[col].fillna("")
    else:
        return pd.Series([""] * len(gdf))

import pandas as pd

parks = gdf[
    gdf.get("leisure", "").fillna("").isin(["park"]) |
    gdf.get("landuse", "").fillna("").isin(["recreation_ground"])
]

buildings = gdf[get_column_safe(gdf, "building") != ""]
streets = gdf[get_column_safe(gdf, "highway") != ""]

print("Features extracted:")
print("Parks:", len(parks))
print("Buildings:", len(buildings))
print("Streets:", len(streets))

# -------------------------------
# 4️⃣ Save GeoPackages
# -------------------------------
output_folder = "san_francisco_layers"
os.makedirs(output_folder, exist_ok=True)

parks_fp = os.path.join(output_folder, "parks.gpkg")
buildings_fp = os.path.join(output_folder, "buildings.gpkg")
streets_fp = os.path.join(output_folder, "streets.gpkg")

parks.to_file(parks_fp, layer="parks", driver="GPKG")
buildings.to_file(buildings_fp, layer="buildings", driver="GPKG")
streets.to_file(streets_fp, layer="streets", driver="GPKG")

print(f"Saved layers to {output_folder}")

# -------------------------------
# 5️⃣ Load layers into QGIS and apply styles
# -------------------------------
from qgis.core import QgsProject, QgsVectorLayer, QgsFillSymbol, QgsLineSymbol, QgsSymbol

project = QgsProject.instance()

# Function to load layer and apply style
def load_and_style(gpkg_path, layer_name, geom_type, style_dict):
    vlayer = QgsVectorLayer(f"{gpkg_path}|layername={layer_name}", layer_name, "ogr")
    if not vlayer.isValid():
        print(f"Failed to load {layer_name}")
        return None
    project.addMapLayer(vlayer)
    
    if geom_type == "polygon":
        symbol = QgsFillSymbol.createSimple({
            "color": style_dict["fill_color"],
            "outline_color": style_dict.get("outline_color", "#000000"),
            "outline_width": str(style_dict.get("outline_width", 0.26)),
            "style": "solid"
        })
        vlayer.renderer().setSymbol(symbol)
    elif geom_type == "line":
        symbol = QgsLineSymbol.createSimple({
            "color": style_dict["line_color"],
            "width": str(style_dict.get("line_width", 0.5))
        })
        vlayer.renderer().setSymbol(symbol)
    vlayer.triggerRepaint()
    return vlayer

# Define styles
park_style = {"fill_color": "#78C679", "outline_color": "#4B8B3B", "outline_width": 0.3}
building_style = {"fill_color": "#B0B0B0", "outline_color": "#666666", "outline_width": 0.2}
street_style = {"line_color": "#333333", "line_width": 0.8}

# Load and style
load_and_style(parks_fp, "parks", "polygon", park_style)
load_and_style(buildings_fp, "buildings", "polygon", building_style)
load_and_style(streets_fp, "streets", "line", street_style)

print("All layers loaded and styled in QGIS!")


✅ Features of this script

Automatic download of San Francisco .osm.pbf with size check.

Safe tag extraction (avoids KeyErrors).

GeoPackage export for easy QGIS import.

Automatic loading into QGIS using PyQGIS.

Custom styling applied (green parks, gray buildings, black streets).

💡 How to run

Option A: QGIS Python Console

Open QGIS → Plugins → Python Console

Copy-paste the script and run.

Option B: Standalone QGIS script

Save as san_francisco_qgis.py and run with QGIS Python environment:

qgis_process run python3:san_francisco_qgis.py
